# Make final Blueprint and corridor layers
Combines the full surface corridor outputs with the zonation results to determine corridor cut offs and make final corridors and blueprint

Created by Amy Keister, last run by Amy Keister on August 14, 2022. It took 3 hours to run

In [1]:
import os
import arcpy
import numpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace = 
OutWorkspace = r"D:\SE_Blueprint_2023\10_AddCorridors\AddCorridors2023.gdb"
OutWorkspace2 = r"D:\SE_Blueprint_2023\10_AddCorridors\Final2023"

In [4]:
# define final outputs
IC = r"D:\SE_Blueprint_2023\10_AddCorridors\Final2023\ContinentalInlandCorridors2023.tif"
MC = r"D:\SE_Blueprint_2023\10_AddCorridors\Final2023\ContinentalMarineCorridors2023.tif"
# in the Caribbean we didn't separate marine from inland, we ran corridors across all
VC = r"D:\SE_Blueprint_2023\10_AddCorridors\Final2023\CaribbeanCorridors2023.tif"
Blue = r"D:\SE_Blueprint_2023\10_AddCorridors\Final2023\Blueprint2023.tif"

In [5]:
# define rasters used for cell size, extent, and snapping
SEraster= r"D:\SE_Blueprint_2023\2_FullExtent\SEBlueprint2023ExtentVIPR6.tif"

In [6]:
# define additional inputs
#subR= r"G:\GIS_DATA\SALCC_BLUEPRINT\Version_2021\Blueprint_2021_Download\1_ExtentLayers\SouthAtlantic2021Subregions.shp"
Zonation= r"D:\SE_Blueprint_2023\7_CombineZonation\CombineZonation.tif"

# this is the way we separate inland from marine on the Continental portion for the purposes of corridor analysis
Inland= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\A_ConInlandResistance.gdb\InlandNearShoreBuffRNoEst"

# this is the VIPR extent, since we don't separate marine/inland in the VIPR area, we use this to mask
VIPRe= r"D:\SE_Blueprint_2023\1_VIPR_Extent\VIPR_Extent_v6.tif"

InlandHub= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ba_Con_LM_Input.gdb\Hubs"
InlandCorr= r"D:\SE_Blueprint_2023\9_Linkagemapper\ContinentalInland\ConInlandClipLMOutputToHex.gdb\Mosaic2"

VIPRHub= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ha_VIPR_LM_Input.gdb\Hubs"
VIPRCorr= r"D:\SE_Blueprint_2023\9_Linkagemapper\VIPR\LM1\output\corridors.gdb\LM1_corridors"

EMarineHub= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ea_ConMarine_LM_Input.gdb\Hubs"
EMarineCorr= r"D:\SE_Blueprint_2023\9_Linkagemapper\ConMarine\ConMarineClipLMOutputToSub.gdb\Mosaic2"

# reservoir estimate with reservoirs as 0 and everything else as 1
ResNull= r"D:\BaseBlueprint\2022\2_Reservoirs\BaseBP_2022_ResNull.tif"

# Start Analysis

In [7]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [8]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\10_AddCorridors\AddCorridors2023.gdb


# Continental inland

## inland hubs
bring in inland hubs

We need to make all hubs "medium" priority, if they aren't already, so that we don't have corridors going to no where. 

In 2021, we had a new problem with inland hubs created by the new Piedmont Prairie indicator. This indicator added some areas that are 15 miles long and only 30 meters wide to some hubs. We have to perform the corridor analysis at 90 meter resolution due to memory issues with Linkage Mapper 2.0.0 (which may get better with Linkage Mapper 3.1.0). During a linkage mapper run, the hubs are converted to rasters with the run resolution. This wasn't a problem before because our hubs did not have these very long very think protuberances. But now, that conversion splits the hubs with these protuberances into a bunch of small, non-contiguous pixles.This throws an error in Linkage Mapper. 

To address this I tried some more complicated things to try to retain the protuberances, but I ran out of time before I got it to work.

In the end I just resampled the hubs to 90 meter pixels myself, which removed the protuberances (because they were no longer a part of the patch/clump for the hub and did not meet the 2,000 acre minimum size. Linkage Mapper requires a polygon input for corridors though, so for that purpose, I converted the hubs back to vector. These steps are done in the other code B_ContinentalInlandHubs.ipynb

We are keeping these steps in case this happens again. Be aware the hubs look weird due to this.

In [9]:
# make a copy of hubs
arcpy.management.CopyFeatures(InlandHub, "ContinentalInlandHubs2023", '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\AddCorridors2023.gdb\\ContinentalInlandHubs2023'>

In [10]:
# add and calculate a field for converting hubs to rasters
arcpy.management.CalculateField("ContinentalInlandHubs2023", "raster", 1, "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\AddCorridors2023.gdb\\ContinentalInlandHubs2023'>

In [11]:
# convert hubs to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent=SEraster):
    arcpy.conversion.PolygonToRaster("ContinentalInlandHubs2023", "raster", "InlandHubR", "CELL_CENTER", "NONE", SEraster)

In [12]:
# make NoData values 0
out_raster = arcpy.sa.Reclassify("InlandHubR", "Value", "NODATA 0", "DATA"); out_raster.save("InlandHub0")

In [13]:
# export inland hubs for final data
arcpy.conversion.FeatureClassToShapefile("ContinentalInlandHubs2023", OutWorkspace2)

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\Final2023'>

## zonation
bring in zonation results and limit to inland areas

The zonation results I'm bringing in have already been converted to integer, this was necessary so that we could rebalance the inland contenential zonation to account for the masks that we use in zonation 4

In [14]:
# limit zonation results to inland areas
out_raster = arcpy.sa.ExtractByMask(Zonation, Inland); out_raster.save("ZInland")

In [15]:
# use a conditional statement to identify areas that did not score high enough to be a priority (<49) and 
# give them a value of 0, keep the other values the same
out_raster = arcpy.sa.Con("ZInland", 0, "ZInland", "Value < 49"); out_raster.save("INoPri")

In [16]:
# use a conditional statement to identify areas that are the highest priority (>89) and 
# give them a value of 400, keep the other values the same

# I'm chaning to value > 90 to match hubs

out_raster = arcpy.sa.Con("INoPri", 400, "INoPri", "Value > 89"); out_raster.save("INoPriH")

In [17]:
# use a conditional statement to identify areas that are the high priority (>74 and <400) and 
# give them a value of 300, keep the other values the same as above
out_raster = arcpy.sa.Con("INoPriH", 300, "INoPriH", "Value > 74 And Value < 400"); out_raster.save("INoPriHH")

The pixels that are still in question are the "medium" and "corridors."  

The starting point for "medium" is the zonation results from 50 - 74.  However, we want 5% to be corridors, so really Medium is 55 - 74 however, we also need to add in any secured areas that aren't already medium, so we may need to change that minimum value to make room for the extra secured areas, depending on how many were not already covered by other Highest, High, and Medium pixels. 

I'm going to try a lower bound of >55,this will give me values of 56-74 as medium, which should already cover around 19% then I'll have to add in the hubs not already covered, which have historically been around 1% 

I need to check this output to see how much of the area is covered by values of 1, those are hubs not already covered if it is more than 1%, I'll need to change the boundary this will be as close as get to giving corridors ~5% of the total area the total priority should add to 50%

In [18]:
# bring in inland hubs that aren't captured already, give them a value of 1, otherwise, retain the value of the input
out_raster = arcpy.sa.Con("INoPriHH", "INoPriHH", "InlandHub0", "Value > 55"); out_raster.save("INoPriHH55Hubs")

In [19]:
# give reservoirs a value of 0
out_raster = arcpy.sa.Times("INoPriHH55Hubs",ResNull); out_raster.save("INoPriHH55HubsRes")

## corridors
bring in inland full corridor surface

In [20]:
# clip to inland area
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent=SEraster):
    out_raster = arcpy.sa.ExtractByMask(InlandCorr, SEraster); out_raster.save("InlandCorrSE")

In [21]:
# test different thresholds to see if I can get close to 5% corridors. The part of the 
# corridor that intersects with the Highest, High, and Medium doesn't count to the 5%
# I'm going to start the test with values < 70,000, based on last year's result

out_raster = arcpy.sa.Con("InlandCorrSE", 100, 0, "Value < 61000"); out_raster.save("InlandCorr1")

In [22]:
# remove reservoirs from corridors
# do this later because low is good in the corridor output
out_raster = arcpy.sa.Times("InlandCorr1",ResNull); out_raster.save("InlandCorr1Res")

In [23]:
# make NoData values 0
with arcpy.EnvManager(outputCoordinateSystem=sr, mask=SEraster):
    out_raster = arcpy.sa.Reclassify("InlandCorr1Res", "Value", "NODATA 0", "DATA"); out_raster.save("InlandCorr1Res0")

## combine zonation w/ corridors
combine zonation with inland corridors

In [24]:
# fill in the above corridor test into the zonation so I can test to see if I get the total of the 0 values to 50%
# this would mean that all the non 0 values add up to 50%
# check the count on the 0 values, divide by the sum of the count, check if it is 50%
# on the 78,000 threshold, the 0 values in this output cover 50.0% of the area this is good
out_raster = arcpy.sa.Con("INoPriHH55HubsRes", "INoPriHH55HubsRes", "InlandCorr1Res0", "Value > 0"); out_raster.save("INoPriHH55HubsResCorr")

I take the attribute values from the above raster and put it in excel

I take a sum of the count field, then divide the count for each class by that sum, to get percent area in each class

- I want the class with values of 0 to equal 50%
- I want the class with values of 100 to equal 5%
- I want the class with values of 300 to equal 15%
- I want the class with values of 400 to equal 10%

- If the corridor class is more than 5%, I reduce the corridor threshold above and try again
- If the corridor class is less than 5%, I increase the corridor threshold above and try again

need to write code for this, in the short term I've just been doing this by hand

In [25]:
# Change all those medium values to 200
# this has all the info we need for inland blueprint, just needs to be reclassified
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent=SEraster, mask=SEraster):
    out_raster = arcpy.sa.Con("INoPriHH55HubsResCorr", 200, "INoPriHH55HubsResCorr", "Value > 0 AND Value < 100"); out_raster.save("INoPriHH55HubsResCorrM")

### Just to double check, I take the attribute values from the above raster and put it in excel

I take a sum of the count field, then divide the count for each class by that sum, to get percent area in each class

- I want the class with values of 0 to equal 50%
- I want the class with values of 100 to equal 5%
- I want the class with values of 200 to equal 20%
- I want the class with values of 300 to equal 15%
- I want the class with values of 400 to equal 10%

need to write code for this, in the short term I've just been doing this by hand

In [26]:
# reclass corridors for export
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent=SEraster, mask=SEraster):
    out_raster = arcpy.sa.Con("InlandCorr1Res0", 1, '', "Value = 100"); out_raster.save("InlandCorr1")

In [27]:
# export corridors, now that we know the threshold is as good as we can get
arcpy.management.CopyRaster("InlandCorr1", IC, '', None, "3", "NONE", "NONE", "2_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\Final2023\\ContinentalInlandCorridors2023.tif'>

# Continental estuarine/marine

## estuarine/marine hubs
bring in estuarine/marine hubs

In [28]:
# make a copy of hubs
arcpy.management.CopyFeatures(EMarineHub, "EMarineHub", '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\AddCorridors2023.gdb\\EMarineHub'>

In [29]:
# add and calculate a field for converting hubs to rasters
arcpy.management.CalculateField("EMarineHub", "raster", 1, "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\AddCorridors2023.gdb\\EMarineHub'>

In [30]:
# because we created these 90 meter resistance raster seperately and didn't specify something to snap to, they happened
# to not snap together. I'm going to try to get them to snap by forcing the marine corrdior outputs to snap to the inland 90 meter raster
Rextent90= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ba_Con_LM_Input.gdb\Resistance"

In [31]:
# convert hubs to 90 meter raster and snap to inland 90 meter resistance raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent90, cellSize=Rextent90):
    arcpy.conversion.PolygonToRaster("EMarineHub", "raster", "EMarineHubR90", "CELL_CENTER", "NONE", Rextent90)

In [32]:
# convert re-snaped hubs back to vector
arcpy.conversion.RasterToPolygon("EMarineHubR90", "EstuarineAndMarineHubs2023", "NO_SIMPLIFY", "Value", "MULTIPLE_OUTER_PART", None)

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\AddCorridors2023.gdb\\EstuarineAndMarineHubs2023'>

In [33]:
# add and calculate a field for converting hubs to rasters
arcpy.management.CalculateField("EstuarineAndMarineHubs2023", "raster", 1, "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\AddCorridors2023.gdb\\EstuarineAndMarineHubs2023'>

In [34]:
# convert hubs to 30 meter raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent=SEraster):
    arcpy.conversion.PolygonToRaster("EstuarineAndMarineHubs2023", "raster", "EMarineHubR", "CELL_CENTER", "NONE", SEraster)

In [35]:
# make NoData values 0
out_raster = arcpy.sa.Reclassify("EMarineHubR", "Value", "NODATA 0", "DATA"); out_raster.save("EMarineHub0")

In [36]:
# because the inland hubs and resistance didn't happen to snap the same way as the marine hubs and resistance when
# we converted them seperately they are shifted 30 meters. I'm hacking a way to take that shift away. This is not ideal
# and it also won't be a total fix. But at least we won't have the 30 meter shift


In [37]:
# export inland hubs for final data
arcpy.conversion.FeatureClassToShapefile("EstuarineAndMarineHubs2023", OutWorkspace2)

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\Final2023'>

## zonation
bring in zonation results and limit them to estuarine/marine areas

The zonation results I'm bringing in have already been converted to integer, this was necessary for the Contenintal inland portion so that we could rebalance due to the zonation 4 mask. I went ahead and did it for all zonation results in 2023, but may want to rethink this when we switch all our analysis to zonation 5

In [38]:
# reclassify to make estuarine/marine mask 
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent=SEraster, mask=SEraster):
    out_raster = arcpy.sa.Reclassify(Inland, "Value", "NODATA 1;1 NODATA", "DATA"); out_raster.save("EMarine")

In [39]:
# limit zonation results to estuarine/marine areas
out_raster = arcpy.sa.ExtractByMask(Zonation, "EMarine"); out_raster.save("ZEMarine")

In [40]:
# use a conditional statement to identify areas that did not score high enough to be a priority (<49) and 
# give them a value of 0, keep the other values the same
# for marine, I'm using <55 since I don't have extra secured area hubs, I tested a few and this seemed to work
out_raster = arcpy.sa.Con("ZEMarine", 0, "ZEMarine", "Value < 57"); out_raster.save("MNoPri")

In [41]:
# use a conditional statement to identify areas that are the highest priority (>89) and 
# give them a value of 400, keep the other values the same
out_raster = arcpy.sa.Con("MNoPri", 400, "MNoPri", "Value > 90"); out_raster.save("MNoPriH")

In [42]:
# use a conditional statement to identify areas that are the high priority (>74 and <400) and 
# give them a value of 300, keep the other values the same as above
out_raster = arcpy.sa.Con("MNoPriH", 300, "MNoPriH", "Value > 75 And Value < 400"); out_raster.save("MNoPriHH")

## corridors
bring in estuarine/marine corridors

In [43]:
# test different thresholds to see if I can get close to 5% corridors. The part of the 
# corridor that intersects with the Highest, High, and Medium doesn't count to the 5%
# I'm going to start the test with values < 78,000, based on last year's result
# run the steps below to test
#
#<750,000 gave me 5.6% corridors, need a lower threshold,
#<700,000 gave me 5.3% corridors, need a lower threshold,
#<580,000 game me 
#
# be aware that I have 11% in higest and 26% in high here, I think it is okay because this is adding in
# estuaries all along the gulf, and priorities tend to be higer along the coast.
out_raster = arcpy.sa.Con(EMarineCorr, 100, 0, "Value < 360000"); out_raster.save("EMarineCorr1")

In [44]:
# make NoData values 0
with arcpy.EnvManager(outputCoordinateSystem=sr, mask=SEraster):
    out_raster = arcpy.sa.Reclassify("EMarineCorr1", "Value", "NODATA 0", "DATA"); out_raster.save("EMarineCorr10")

## combine zonation w/ corridors
combine zonation with estuarine/marine corridors

In [45]:
# fill in the above corridor test into the zonation so I can test to see if I get the total of the 0 values to 50%
# this would mean that all the non 0 values add up to 50%
# check the count on the 0 values, divide by the sum of the count, check if it is 50%
# on the 78,000 threshold, the 0 values in this output cover 50.0% of the area this is good
out_raster = arcpy.sa.Con("MNoPriHH", "MNoPriHH", "EMarineCorr10", "Value > 0"); out_raster.save("MNoPriHHCorr")

need to write code that calculates the sum of the count field in the above raster, then divides the count for the values that = 0 by the sum of the count. Want it to be 50%

I take the attribute values from the above raster and put it in excel. I take a sum of the count field, then divide the count for each class by that sum, to get percent area in each class

- I want the class with values of 0 to equal 50%
- I want the class with values of 100 to equal 5%
- I want the class with values of 300 to equal 15%
- I want the class with values of 400 to equal 10%

- If the corridor class is more than 5%, I reduce the corridor threshold above and try again
- If the corridor class is less than 5%, I increase the corridor threshold above and try again

need to write code for this, in the short term I've just been doing this by hand

In [46]:
# Change all those medium values to 200
# this has all the info we need for inland blueprint, just needs to be reclassified
out_raster = arcpy.sa.Con("MNoPriHHCorr", 200, "MNoPriHHCorr", "Value > 0 AND Value < 100"); out_raster.save("MNoPriHHCorrM")

I take the attribute values from the above raster and put it in excel. I take a sum of the count field, then divide the count for each class by that sum, to get percent area in each class

- I want the class with values of 0 to equal 50%
- I want the class with values of 100 to equal 5%
- I want the class with values of 200 to equal 20%
- I want the class with values of 300 to equal 15%
- I want the class with values of 400 to equal 10%

need to write code for this, in the short term I've just been doing this by hand

In [47]:
# reclass corridors for export
out_raster = arcpy.sa.Con("EMarineCorr10", 1, '', "Value = 100"); out_raster.save("EMarineCorr10a1")

In [48]:
# export corridors, now that we know the threshold is as good as we can get
arcpy.management.CopyRaster("EMarineCorr10a1", MC, '', None, "3", "NONE", "NONE", "2_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\Final2023\\ContinentalMarineCorridors2023.tif'>

# Caribbean

## Caribbean hubs
bring in Caribbean hubs

In [49]:
# make a copy of hubs
arcpy.management.CopyFeatures(VIPRHub, "CaribbeanHubs2023", '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\AddCorridors2023.gdb\\CaribbeanHubs2023'>

In [50]:
# add and calculate a field for converting hubs to rasters
arcpy.management.CalculateField("CaribbeanHubs2023", "raster", 1, "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\AddCorridors2023.gdb\\CaribbeanHubs2023'>

In [51]:
# convert hubs to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent=VIPRe):
    arcpy.conversion.PolygonToRaster("CaribbeanHubs2023", "raster", "VIPRHubR", "CELL_CENTER", "NONE", SEraster)

In [52]:
# make NoData values 0
out_raster = arcpy.sa.Reclassify("VIPRHubR", "Value", "NODATA 0", "DATA"); out_raster.save("VIPRHub0")

In [53]:
# export inland hubs for final data
arcpy.conversion.FeatureClassToShapefile("CaribbeanHubs2023", OutWorkspace2)

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\Final2023'>

## zonation
bring in zonation results and limit to Caribbean areas

The zonation results I'm bringing in have already been converted to integer, this was necessary for the Contenintal inland portion so that we could rebalance due to the zonation 4 mask. I went ahead and did it for all zonation results in 2023, but may want to rethink this when we switch all our analysis to zonation 5


In [54]:
# limit zonation results to Caribbean areas
out_raster = arcpy.sa.ExtractByMask(Zonation, VIPRe); out_raster.save("Zvipr")

In [55]:
# use a conditional statement to identify areas that did not score high enough to be a priority (<49) and 
# give them a value of 0, keep the other values the same
out_raster = arcpy.sa.Con("Zvipr", 0, "Zvipr", "Value < 49"); out_raster.save("VNoPri")

In [56]:
# use a conditional statement to identify areas that are the highest priority (>89) and 
# give them a value of 400, keep the other values the same
out_raster = arcpy.sa.Con("VNoPri", 400, "VNoPri", "Value > 89"); out_raster.save("VNoPriH")

In [57]:
# use a conditional statement to identify areas that are the high priority (>74 and <400) and 
# give them a value of 300, keep the other values the same as above
out_raster = arcpy.sa.Con("VNoPriH", 300, "VNoPriH", "Value > 74 And Value < 400"); out_raster.save("VNoPriHH")

In [58]:
# bring in inland hubs that aren't captured already, give them a value of 1, otherwise, retain the value of the input
out_raster = arcpy.sa.Con("VNoPriHH", "VNoPriHH", "VIPRHub0", "Value > 54"); out_raster.save("VNoPriHH55Hubs")

## corridors
bring in Caribbean full corridor surface

In [59]:
# test different thresholds to see if I can get close to 5% corridors. The part of the 
# corridor that intersects with the Highest, High, and Medium doesn't count to the 5%
# I'm going to start the test with values < 70,000, based on last year's result

out_raster = arcpy.sa.Con(VIPRCorr, 100, 0, "Value < 27500"); out_raster.save("VIPRCorr1")

In [60]:
# make NoData values 0
with arcpy.EnvManager(outputCoordinateSystem=sr, mask=SEraster):
    out_raster = arcpy.sa.Reclassify("VIPRCorr1", "Value", "NODATA 0", "DATA"); out_raster.save("VIPRCorr10")

## combine zonation w/ corridors
combine zonation with Caribbean corridors

In [61]:
# fill in the above corridor test into the zonation so I can test to see if I get the total of the 0 values to 50%
# this would mean that all the non 0 values add up to 50%
# check the count on the 0 values, divide by the sum of the count, check if it is 50%
# on the 78,000 threshold, the 0 values in this output cover 50.0% of the area this is good
out_raster = arcpy.sa.Con("VNoPriHH55Hubs", "VNoPriHH55Hubs", "VIPRCorr10", "Value > 0"); out_raster.save("VNoPriHH55HubsCorr")

I take the attribute values from the above raster and put it in excel. I take a sum of the count field, then divide the count for each class by that sum, to get percent area in each class

- I want the class with values of 0 to equal 50%
- I want the class with values of 100 to equal 5%
- I want the class with values of 300 to equal 15%
- I want the class with values of 400 to equal 10%

- If the corridor class is more than 5%, I reduce the corridor threshold above and try again
- If the corridor class is less than 5%, I increase the corridor threshold above and try again

need to write code for this, in the short term I've just been doing this by hand

In [62]:
# Change all those medium values to 200
# this has all the info we need for inland blueprint, just needs to be reclassified
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent=VIPRe, mask=SEraster):
    out_raster = arcpy.sa.Con("VNoPriHH55HubsCorr", 200, "VNoPriHH55HubsCorr", "Value > 0 AND Value < 100"); out_raster.save("VNoPriHH55HubsCorrM")

In [63]:
# reclass corridors for export
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent=VIPRe, mask=SEraster):
    out_raster = arcpy.sa.Con("VIPRCorr10", 1, '', "Value = 100"); out_raster.save("VCorr1")

In [64]:
# export corridors, now that we know the threshold is as good as we can get
arcpy.management.CopyRaster("VCorr1", VC, '', None, "3", "NONE", "NONE", "2_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\Final2023\\CaribbeanCorridors2023.tif'>

# Make the final Blueprint
Combine Continental inland, Continental estuarine/marine, and Caribbean data

In [65]:
# combine inland and estuarine/marine data using cell statistics maximum
# combine rasters, keeping max value
# combine rasters, keeping max value
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent=SEraster, mask=SEraster):
    out_raster = arcpy.sa.CellStatistics(["INoPriHH55HubsResCorrM", "MNoPriHHCorrM","VNoPriHH55HubsCorrM"],"MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("IMVa")

In [66]:
# reclassify to final blueprint values
out_raster = arcpy.sa.Reclassify("IMVa", "Value", "0 0;100 1;200 2;300 3;400 4", "DATA"); out_raster.save("IMV")

In [67]:
# add multiple fields to hold descriptions and colors
arcpy.management.AddFields("IMV", field_description=[["Descript", "TEXT", "", "255", "", ""], ["Red", "SHORT", "", "", "", ""], ["Green", "SHORT", "", "", "", ""], ["Blue", "SHORT", "", "", "", ""]])[0]

'D:\\SE_Blueprint_2023\\10_AddCorridors\\AddCorridors2023.gdb\\IMV'

In [68]:
# set code block for next step
codeblock = """
def Reclass(Value):
	if Value == 4:
		return 'Highest priority'
	if Value == 3:
		return 'High priority'
	if Value == 2:
		return 'Medium priority'
	if Value == 1:
		return 'Priority connections'
	else:
		return 'Lower priority'
		
def Reclass1(Value):
	if Value == 4:
		return 77
	if Value == 3:
		return 132
	if Value == 2:
		return 140
	if Value == 1:
		return 108
	else:
		return 255
		
def Reclass2(Value):
	if Value == 4:
		return 0
	if Value == 3:
		return 63
	if Value == 2:
		return 150
	if Value == 1:
		return 108
	else:
		return 255
		
def Reclass3(Value):
	if Value == 4:
		return 75
	if Value == 3:
		return 152
	if Value == 2:
		return 198
	if Value == 1:
		return 108
	else:
		return 255
"""

# calculate description field
arcpy.management.CalculateField("IMV", "Descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")
# calculate Red field
arcpy.management.CalculateField("IMV", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("IMV", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("IMV", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\AddCorridors2023.gdb\\IMV'>

In [71]:
# export as .tif 
arcpy.management.CopyRaster("IMV", Blue, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

<Result 'D:\\SE_Blueprint_2023\\10_AddCorridors\\Final2023\\Blueprint2023.tif'>

In [70]:
end = time.time()
print(end - start)

10860.583675146103
